# Sizing optimization under uncertainty

In [ ]:
# Install hydesign if needed
import importlib
if not importlib.util.find_spec("hydesign"):
    !pip install git+https://gitlab.windenergy.dtu.dk/TOPFARM/hydesign.git   

In [1]:
from hydesign.assembly.hpp_assembly import hpp_model 
from hydesign.assembly.hpp_assembly_reliability import ReliabilityModel 
from hydesign.Parallel_EGO import EfficientGlobalOptimizationDriver
import os
import pandas as pd
from hydesign.examples import examples_filepath
from hydesign.utils import sample_mean

HyDesign can run different realizations of up and down time of individual components throughout the lifetime. These can be aggregated together and utilized to perform sizing optimization under uncertainty.
In this example we will include inverter and PV panel sizes as design variables.

In [2]:
name = 'France_good_wind'
examples_sites = pd.read_csv(f'{examples_filepath}examples_sites.csv', index_col=0, sep=';')
ex_site = examples_sites.loc[examples_sites.name == name]

longitude = ex_site['longitude'].values[0]
latitude = ex_site['latitude'].values[0]
altitude = ex_site['altitude'].values[0]

sim_pars_fn = examples_filepath+ex_site['sim_pars_fn'].values[0]
input_ts_fn = examples_filepath+ex_site['input_ts_fn'].values[0]

inputs = {
    # HPP Model Inputs
    'name': name,
    'longitude': longitude,
    'latitude': latitude,
    'altitude': altitude,
    'input_ts_fn': input_ts_fn,
    'sim_pars_fn': sim_pars_fn,
    'num_batteries': 10,
    'work_dir': './',
    'hpp_model': ReliabilityModel,

    # EGO Inputs
    'opt_var': "NPV_over_CAPEX",
    'n_procs': 4,
    'n_doe': 10,
    'n_clusters': 4, # total number of evals per iteration = n_clusters + 2*n_dims
    'n_seed': 0,
    'max_iter': 3,
    'final_design_fn': 'hydesign_design_0.csv',
    'npred': 2e4,
    'tol': 1e-6,
    'min_conv_iter': 3,

    # Reliability Model Inputs
    'reliability_data_set_path': os.path.join(examples_filepath, 'reliability'),
    'battery_ds_fn': 'reliability_data_set_BESS.nc',
    'transformer_ds_fn': 'reliability_data_set_transformer.nc',
    'inverter_ds_fn': 'reliability_data_set_inverter.nc',
    'PV_ds_fn': 'reliability_data_set_PV.nc',
    'WT_ds_fn': 'reliability_data_set_WT.nc',
    'n_reliability_seed': 2,
    'aggregation_method': sample_mean,
    'reliability_hpp_model': hpp_model,
    
    # Design Variables
    'variables': {
        'clearance [m]': {
            'var_type':'fixed',
            'value': 10
            },
        'sp [W/m2]': {
            'var_type':'fixed',
            'value': 350,
            },
        'p_rated [MW]': {
            'var_type':'fixed',
            'value': 6
            },
        'Nwt': {
            'var_type':'fixed',
            'value': 200
            },
        'wind_MW_per_km2 [MW/km2]': {
            'var_type':'fixed',
            'value': 7
            },
        'solar_MW [MW]': {
            'var_type':'fixed',
            'value': 200
            },
        'surface_tilt [deg]': {
            'var_type':'fixed',
            'value': 25
            },
        'surface_azimuth [deg]': {
            'var_type':'design',
            'limits':[150, 210],
            'types':'float'
            },
        'DC_AC_ratio': {
            'var_type':'fixed',
            'value':1.0,
            },
        'b_P [MW]': {
            'var_type':'fixed',
            'value': 50
            },
        'b_E_h [h]': {
            'var_type':'fixed',
            'value': 6
            },
        'cost_of_battery_P_fluct_in_peak_price_ratio': {
            'var_type':'fixed',
            'value': 10
            },
        'inverter_size [kW]': {
            'var_type':'design',
            'limits':[100, 2000],
            'types':'float'
            },
        'panel_size [W]': {
            'var_type':'design',
            'limits':[100, 500],
            'types':'float'
            },
        },
    }



In [3]:
EGOD = EfficientGlobalOptimizationDriver(**inputs)
EGOD.run()
result = EGOD.result






Sizing a HPP plant at France_good_wind:


Fixed parameters on the site
-------------------------------
longitude = -0.864258
latitude = 48.744116
altitude = 302.0



Initial 10 simulations took 1.87 minutes
  Current solution -NPV_over_CAPEX = 1.765E-01
  Current No. model evals: 10

Update sm and extract candidate points took 0.15 minutes
Check-optimal candidates: new 10 simulations took 2.48 minutes
  Current solution -NPV_over_CAPEX = 1.763E-01
  Current No. model evals: 20
  rel_yopt_change = 1.21E-03
Iteration 1 took 2.64 minutes

Update sm and extract candidate points took 0.19 minutes
Check-optimal candidates: new 10 simulations took 2.49 minutes
  Current solution -NPV_over_CAPEX = 1.763E-01
  Current No. model evals: 29
  rel_yopt_change = 4.62E-06
Iteration 2 took 2.7 minutes

Update sm and extract candidate points took 0.15 minutes
Check-optimal candidates: new 10 simulations took 2.16 minutes
  Current solution -NPV_over_CAPEX = 1.763E-01
  Current No. model evals: 39
 